In [9]:
import pandas as pd

df = pd.DataFrame({
    'Animal' : ['monkey', 'parrot', 'cow'],
    'HowCute' : [100, 80, 60],
    'Weight' : [10, 1, 150]
})

#pandas는 순서가 다르다
print(f"원본 데이터의 메모리 주소 : {id(df)}")
print(df.loc[1, 'Animal']) #parrot
print(df.iloc[0,1]) #100


원본 데이터의 메모리 주소 : 2428568325968
parrot
100


In [10]:
#단순 할당(reference)
df_reference = df
print(f"단순 할당한 데이터의 메모리 주소 : {id(df_reference)}")
    #주소가 같다 = 하나를 바꾸면 같이 바뀐다.

#슬라이싱(view)
df_view = df.iloc[:]
print(f"슬라이싱한 데이터의 메모리 주소 : {id(df_view)}")
print("base메모리와 view메모리 비교")
print(df['Animal'].values.base is df_view['Animal'].values.base)  #True
    #객체 ID는 다르다.(껌데기가 다름)
    #but base내부메모리는 같다. (데이터를 공유)

df.loc[0, "HowCute"] = 50
df_view.loc[0, "HowCute"] = 700
print("@@@@@@@@@@@@@@@@@@@")
print(df.iloc[0,1])
print(df_view.iloc[0,1])



#복사(copy)
df_copy = df.copy()
print(f"복사한 데이터의 메모리 주소 : {id(df_copy)}")
    #주소가 다르다 = 데이터가 독립적으로 존재한다.
df.loc[1, 'HowCute'] = 100
df_copy.loc[1, 'HowCute'] = 200
print("@@@@@@@@@@@@@@@@@@@")
print(df.iloc[1, 1])
print(df_copy.iloc[1, 1])

단순 할당한 데이터의 메모리 주소 : 2428568325968
슬라이싱한 데이터의 메모리 주소 : 2428501696592
base메모리와 view메모리 비교
True
@@@@@@@@@@@@@@@@@@@
700
700
복사한 데이터의 메모리 주소 : 2428568326736
@@@@@@@@@@@@@@@@@@@
100
200


In [11]:
#view copy 추가실습
df2 = pd.DataFrame({
    'Name': ['Kim', 'Lee', 'Park', 'Choi', 'Jung'],
    'Team': ['IT', 'HR', 'IT', 'Marketing', 'HR'],
    'Salary': [5000, 4000, 6000, 4500, 4200],
    'Status': ['Active', 'Active', 'Active', 'Active', 'Leave']
})

print("@@@ 원본 데이터 @@@")
print(df2)

@@@ 원본 데이터 @@@
   Name       Team  Salary  Status
0   Kim         IT    5000  Active
1   Lee         HR    4000  Active
2  Park         IT    6000  Active
3  Choi  Marketing    4500  Active
4  Jung         HR    4200   Leave


In [12]:
#1. chained indexing 경고 테스트
df2[df2['Team'] == 'IT']['Salary'] *= 1.1
print("chained indexing경고 테스트 : IT의 연봉 10% 인상")
print(df2)
    #-> 원본 데이터 안바뀜(copy로 작동)
    # SettingWithCopyWarning 경고뜸 (체이닝 문제)

chained indexing경고 테스트 : IT의 연봉 10% 인상
   Name       Team  Salary  Status
0   Kim         IT    5000  Active
1   Lee         HR    4000  Active
2  Park         IT    6000  Active
3  Choi  Marketing    4500  Active
4  Jung         HR    4200   Leave


C:\Users\bhc02\AppData\Local\Temp\ipykernel_30512\2054329406.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[df2['Team'] == 'IT']['Salary'] *= 1.1


In [13]:
#2. loc를 이용해 chaining 문제 해결
df2.loc[0, 'Salary'] *= 1.2
print("loc(copy)를 쓴 IT연봉 인상")
print(df2) #5000 -> 6000

loc(copy)를 쓴 IT연봉 인상
   Name       Team  Salary  Status
0   Kim         IT    6000  Active
1   Lee         HR    4000  Active
2  Park         IT    6000  Active
3  Choi  Marketing    4500  Active
4  Jung         HR    4200   Leave


In [14]:
#3. copy를 통한 원본 데이터 보존, HR팀만 확인
    # df2_HR = df2.copy(['Team'] == "HR")❌
        #copy는 복사만 하지 조건을 못 붙인다
df2_HR = df2[df2['Team'] == 'HR'].copy()
print(df2_HR)

print("연봉을 모두 0으로 만들기")
df2_HR.loc[:,'Salary'] = 0
print(df2_HR)
print("원본 데이터 값 변경 여부 확인")
print(df2[df2['Team'] == 'HR'])
    #copy가 잘 되어 원본 값은 바뀌지 않았다



   Name Team  Salary  Status
1   Lee   HR    4000  Active
4  Jung   HR    4200   Leave
연봉을 모두 0으로 만들기
   Name Team  Salary  Status
1   Lee   HR       0  Active
4  Jung   HR       0   Leave
원본 데이터 값 변경 여부 확인
   Name Team  Salary  Status
1   Lee   HR    4000  Active
4  Jung   HR    4200   Leave


In [ ]:
#4. inplace 쓰지말자
#status컬럼을 지우려고 한다. inplace가 실제로 메모리 주소를 유지하는지 확인해보자

df2_ref = df2
origin_id = id(df2)
df2.drop('Status', axis=1, inplace=True)
print("기존 id", origin_id)
print("수정 후 id", id(df2))

df2_ref.drop('Salary', axis=1) #재할당 없음 = 날라감~
print(df2_ref)
print(df2)

#결론 : inpalce = True 쓸빠엔 재할당을 해라.


기존 id 2428568327248
수정 후 id 2428568327248
   Name       Team  Salary
0   Kim         IT    6000
1   Lee         HR    4000
2  Park         IT    6000
3  Choi  Marketing    4500
4  Jung         HR    4200
   Name       Team  Salary
0   Kim         IT    6000
1   Lee         HR    4000
2  Park         IT    6000
3  Choi  Marketing    4500
4  Jung         HR    4200
